In [1]:
from nsepy import get_history
from datetime import date
import pandas as pd
from nsepy.derivatives import get_expiry_date

## Obtaining the data

In [13]:
tcs = get_history(symbol='TCS',start=date(2015,1,1),end=date(2015,12,31))

In [8]:
infy = get_history(symbol='INFY',start=date(2015,1,1),end=date(2015,12,31))

In [39]:
# Doesn't work without the index attribute
nifty_data = get_history(symbol="NIFTYIT",start=date(2015,1,1),end=date(2015,12,31),index=True)

In [40]:
print(nifty_data.columns)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Turnover'], dtype='object')


In [53]:
#Making columns consistent for all data
infy_data = infy[['Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']]
tcs_data = tcs[['Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']]

In [52]:
#Checking for consistency in data
print('\n\n\t\t\t\tINFY\n\n\n',infy_data.head(),'\n\n\t\t\t\tTCS\n\n\n',tcs_data.head(),'\n\n\t\t\t\tNIFTY\n\n\n',nifty_data.head())



				INFY


                Open     High     Low    Close   Volume      Turnover
Date                                                                
2015-01-01  1968.95  1982.00  1956.9  1974.40   500691  9.870306e+13
2015-01-02  1972.00  2019.05  1972.0  2013.20  1694580  3.394669e+14
2015-01-05  2009.90  2030.00  1977.5  1995.90  2484256  4.979911e+14
2015-01-06  1980.00  1985.00  1934.1  1954.20  2416829  4.724458e+14
2015-01-07  1965.00  1974.75  1950.0  1963.55  1812479  3.557162e+14 

				TCS


               Open     High      Low    Close   Volume      Turnover
Date                                                                
2015-01-01  2567.0  2567.00  2541.00  2545.55   183415  4.674345e+13
2015-01-02  2551.0  2590.95  2550.60  2579.45   462870  1.188740e+14
2015-01-05  2581.0  2599.90  2524.65  2540.25   877121  2.248886e+14
2015-01-06  2529.1  2529.10  2440.00  2446.60  1211892  2.989615e+14
2015-01-07  2470.0  2479.15  2407.45  2417.70  1318166  3.208362e+14 

				N

## Writing out the data

In [59]:
nifty_data.to_csv('dataset/nifty_data.csv', encoding='utf-8', index=False)

In [60]:
tcs_data.to_csv('dataset/tcs_data.csv', encoding='utf-8', index=False)

In [61]:
infy_data.to_csv('dataset/infy_data.csv', encoding='utf-8', index=False)